In [16]:
from pymongo import MongoClient
import folium
import pandas as pd

In [4]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [12]:
db, coll = connectCollection('datlab','companies_clean')

In [13]:
queryf = list(coll.find())

In [14]:
queryf[0]

{'_id': ObjectId('5dd4011c5b757d36a30c9916'),
 '': 0,
 'name': 'OpenX',
 'category_code': 'advertising',
 'number_of_employees': 305.0,
 'founded_year': 2008,
 'description': 'OpenX',
 'total_money_raised': '$75.5M',
 'latitude': 34.1494713,
 'city': 'Pasadena',
 'location': {'type': 'Point', 'coordinates': [-118.1327468, 34.1494713]}}

In [8]:
def getLocation(office):
    longitude = office['longitude']
    latitude = office['latitude']
    loc = {
        'type':'Point',
        'coordinates':[longitude, latitude]
    }
    return loc

In [9]:
for office in queryf:
    value = {"$set": {'location':getLocation(office)}}
    coll.update_one(office, value)

In [10]:
for office in queryf:
    value = {"$unset": {"longitude":""}}
    coll.update_one(office, value)

In [35]:
for office in queryf:
    value = {"$unset": {"latitude":""}}
    coll.update_one(office, value)

In [26]:
ranking = pd.DataFrame(queryf)
ranking['rank_money'] = ranking['total_money_raised'].rank()
ranking['rank_city'] = ranking['city'].rank()

In [37]:
ranking['total'] = ranking['rank_money'] + ranking['rank_city']

In [40]:
ranking = ranking.sort_values(by='total', ascending = False)

In [42]:
ranking1 = ranking.loc[(ranking['total']>=250)]

In [48]:
ranking1

,_id,,name,category_code,number_of_employees,founded_year,description,total_money_raised,latitude,city,location,rank_money,rank_city,total
5,5dd4011c5b757d36a30c991b,4,Fixya,web,30,2013,Tech Support Community,$8M,37.566879,San Mateo,"{'type': 'Point', 'coordinates': [-122.323895,...",147.5,133.5,281.0
119,5dd4011c5b757d36a30c998d,119,Z2Live,games_video,130,2009,Mobile Games,$6.5M,47.610301,Seattle,"{'type': 'Point', 'coordinates': [-122.3399782...",123.0,144.0,267.0
149,5dd4011c5b757d36a30c99ab,150,Homesnap,analytics,18,2008,online real estate broker,$5.5M,38.960381,Washington,"{'type': 'Point', 'coordinates': [-77.084811, ...",114.0,152.0,266.0
1,5dd4011c5b757d36a30c9917,1,GoingOn,software,40,2008,Private Academic Networks,$8.5M,37.782263,San Francisco,"{'type': 'Point', 'coordinates': [-122.392142,...",142.5,112.0,254.5
16,5dd4011c5b757d36a30c9926,16,Sprout,advertising,18,2008,Build Ads Simply,$8.3M,37.755012,San Francisco,"{'type': 'Point', 'coordinates': [-122.418137,...",140.5,112.0,252.5


5      {'type': 'Point', 'coordinates': [-122.323895,...
119    {'type': 'Point', 'coordinates': [-122.3399782...
149    {'type': 'Point', 'coordinates': [-77.084811, ...
1      {'type': 'Point', 'coordinates': [-122.392142,...
16     {'type': 'Point', 'coordinates': [-122.418137,...
Name: location, dtype: object


In [28]:
map_city=folium.Map(location=[40.7221,-73.9712], zoom_start=12)

In [30]:
for company in queryf:
    folium.Marker(company['location']['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='red'), 
                   ).add_to(map_city)

In [31]:
map_city